<a href="https://colab.research.google.com/github/Dinhthixuanbinh/Note/blob/main/mask_faster_rcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Faster R-cnn

In [ ]:
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import transforms
from PIL import Image

import os
from collections import defaultdict



In [ ]:
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

In [ ]:
!gdown --id 16Gan9NZR53bJsjjLpzb_e5_Q41BktloY
!unzip images_v2_224_112.zip
!rm -rf images_v2_224_112.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=16Gan9NZR53bJsjjLpzb_e5_Q41BktloY
To: /content/images_v2_224_112.zip
100% 1.13M/1.13M [00:00<00:00, 28.5MB/s]
Archive:  images_v2_224_112.zip
  inflating: images_v2/begin_57.jpg  
  inflating: images_v2/end_49.jpg    
  inflating: images_v2/end_57.jpg    
  inflating: images_v2/begin_53.jpg  
  inflating: images_v2/middle_53.jpg  
  inflating: images_v2/begin_52.jpg  
  inflating: images_v2/middle_54.jpg  
  inflating: images_v2/end_51.jpg    
  inflating: images_v2/middle_55.jpg  
  inflating: images_v2/middle_49.jpg  
  inflating: images_v2/end_54.jpg    
  inflating: images_v2/middle_50.jpg  
  inflating: images_v2/middle_58.jpg  
  inflating: images_v2/end_55.jpg    
  inflating: images_v2/begin_50.jpg  
  inflating: i

In [ ]:
fas_model = fasterrcnn_resnet50_fpn(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:02<00:00, 61.5MB/s]


In [ ]:
fas_model.eval()



FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
folder_path = '/content/images_v2'

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])
fas_result =[]
for filename in os.listdir(folder_path):
  image_path = os.path.join(folder_path, filename)
  image = Image.open(image_path)
  image_tensor = transform(image)

  with torch.no_grad():
    predictions = fas_model([image_tensor])
    labels = list(predictions[0]['labels'].numpy())
    pred_class = defaultdict(int)
  for i in labels:
    category_name = COCO_INSTANCE_CATEGORY_NAMES[i]
    pred_class[category_name] += 1
  fas_result.append(pred_class)










# def get_prediction(folder_path, confidence):
#   img = Image.open(folder_path)
#   transform = transforms.Compose([transforms.ToTensor()])
#   image_tensor = transform(image)
#   pred = fas_model([image_tensor])
#   pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].numpy())]
#   pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].detach().numpy())]
#   pred_score = list(pred[0]['scores'].detach().numpy())
#   pred_t = [pred_score.index(x) for x in pred_score if x>confidence][-1]
#   pred_boxes = pred_boxes[:pred_t+1]
#   pred_class = pred_class[:pred_t+1]
#   return pred_boxes, pred_class

# boxes, pred_cls = get_prediction(folder_path, 0.5)

In [ ]:
import csv

csv_file = 'fas_result.csv'
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['name_image', 'results'])

    for filename, label in zip(os.listdir(folder_path), fas_result):
            writer.writerow([filename, label])

### Mask R-cnn

In [ ]:

from torchvision.models.detection import maskrcnn_resnet50_fpn


In [ ]:
mask_model = maskrcnn_resnet50_fpn(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:00<00:00, 215MB/s]


In [ ]:

mask_model.eval()

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])
mask_result =[]
for filename in os.listdir(folder_path):
  image_path = os.path.join(folder_path, filename)
  image = Image.open(image_path)
  image_tensor = transform(image)

  with torch.no_grad():
    predictions = fas_model([image_tensor])
    labels = list(predictions[0]['labels'].numpy())
    pred_class = defaultdict(int)
  for i in labels:
    category_name = COCO_INSTANCE_CATEGORY_NAMES[i]
    pred_class[category_name] += 1
  mask_result.append(pred_class)





In [ ]:
import csv

csv_file = 'mask_result.csv'
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['name_image', 'results'])

    for filename, label in zip(os.listdir(folder_path), mask_result):
            writer.writerow([filename, label])

In [ ]:
# transform = transforms.Compose([transforms.ToTensor()])

In [ ]:
# with torch.no_grad():
#     predictions = mask_model([image_tensor])

